# Group 17 DSCI Project (Section 007)
### Title: Predicting diabetes based on demographic attributes, medical history, and clinical measurements

Darby Quinn #19752633 <br>
Manav Parikh #13928775 <br>
Nitya Goel #89433221 <br>
Reeva Bansal #68061514 <br>

#### Introduction

Diabetes is a condition where the body has issues processing and producing insulin that affects people of all ages. This can cause serious problems such as heart disease, vision loss, and kidney disease. Certain medical factors (e.g. BMI, age, insulin and glucose levels) can help predict whether an individual is likely to have or develop diabetes. 

The question we are answering is: **Can we classify whether or not someone has diabetes based on their blood pressure, BMI and age?**

The dataset we are using to answer this question is a Diabetes dataset found on Kaggle with nine different columns: pregnancies; blood glucose; blood pressure (BP); skin thickness; blood insulinl; body mass index (BMI); diabetes pedigree; age; and outcome. This dataset is pre-divided into training and testing data.

In [2]:
# loading libraries needed to create tables and visualizations 
library(tidyverse)
library(dplyr)

Warning message:
“package ‘ggplot2’ was built under R version 4.3.2”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
